In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [38]:
from google.colab import files
uploaded = files.upload()

Saving shoe-data.csv to shoe-data (3).csv


In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [39]:
df = spark.read.csv('shoe-data (3).csv', header=True)
df.show()

+------+---------+--------------------+----------+
| brand|   gender|            colorway|priceGroup|
+------+---------+--------------------+----------+
|  Nike|    child|Black University ...|      <100|
|  Nike|      men|    White Game Royal|   100-150|
|Jordan|    child|White Tech Grey B...|   150-200|
|Jordan|preschool|White Tech Grey B...|   150-200|
|Jordan|      men|White Tech Grey B...|   150-200|
|Jordan|  toddler|White Tech Grey B...|   150-200|
|Jordan|      men|Black White-Light...|   150-200|
|Jordan|    women| White Bright Citrus|   150-200|
|Jordan|      men|White White-Black...|   150-200|
|Jordan|      men|Hyper Royal Light...|   150-200|
|Jordan|    women|Sail Rust Pink-Wh...|   150-200|
|Jordan|preschool|Varsity Red Black...|      <100|
|Jordan|  toddler|Varsity Red Black...|      <100|
|Jordan|      men|Varsity Red Black...|   150-200|
|Jordan|    child|Anthracite Wolf G...|   100-150|
|Jordan|    child|Varsity Red Black...|   100-150|
|Jordan|      men|White White-M

In [40]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer

In [41]:
tokened = Tokenizer(inputCol="colorway", outputCol="words")
tokened_transformed = tokened.transform(df)
tokened_transformed.show()

+------+---------+--------------------+----------+--------------------+
| brand|   gender|            colorway|priceGroup|               words|
+------+---------+--------------------+----------+--------------------+
|  Nike|    child|Black University ...|      <100|[black, universit...|
|  Nike|      men|    White Game Royal|   100-150|[white, game, royal]|
|Jordan|    child|White Tech Grey B...|   150-200|[white, tech, gre...|
|Jordan|preschool|White Tech Grey B...|   150-200|[white, tech, gre...|
|Jordan|      men|White Tech Grey B...|   150-200|[white, tech, gre...|
|Jordan|  toddler|White Tech Grey B...|   150-200|[white, tech, gre...|
|Jordan|      men|Black White-Light...|   150-200|[black, white-lig...|
|Jordan|    women| White Bright Citrus|   150-200|[white, bright, c...|
|Jordan|      men|White White-Black...|   150-200|[white, white-bla...|
|Jordan|      men|Hyper Royal Light...|   150-200|[hyper, royal, li...|
|Jordan|    women|Sail Rust Pink-Wh...|   150-200|[sail, rust, p

In [42]:
hashing = HashingTF(inputCol='words', outputCol='hashedValues')
hashed_df = hashing.transform(tokened_transformed)
hashed_df.show()

+------+---------+--------------------+----------+--------------------+--------------------+
| brand|   gender|            colorway|priceGroup|               words|        hashedValues|
+------+---------+--------------------+----------+--------------------+--------------------+
|  Nike|    child|Black University ...|      <100|[black, universit...|(262144,[10564,61...|
|  Nike|      men|    White Game Royal|   100-150|[white, game, royal]|(262144,[17821,75...|
|Jordan|    child|White Tech Grey B...|   150-200|[white, tech, gre...|(262144,[10564,75...|
|Jordan|preschool|White Tech Grey B...|   150-200|[white, tech, gre...|(262144,[10564,75...|
|Jordan|      men|White Tech Grey B...|   150-200|[white, tech, gre...|(262144,[10564,75...|
|Jordan|  toddler|White Tech Grey B...|   150-200|[white, tech, gre...|(262144,[10564,75...|
|Jordan|      men|Black White-Light...|   150-200|[black, white-lig...|(262144,[78526,78...|
|Jordan|    women| White Bright Citrus|   150-200|[white, bright, c...

In [43]:
idf = IDF(inputCol='hashedValues', outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [44]:
rescaledData.select('words',"features").show(truncate=False)

+----------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                   |features                                                                                                                                                        |
+----------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[black, university, red, white]         |(262144,[10564,61581,75571,154828],[2.29675032111499,3.5700296732165224,0.9975671324042024,1.1271712596720214])                                                 |
|[white, game, royal]                    |(262144,[17821,75571,138895],[3.754706943702936,0.9975671324042024,5.103960033142478])                                                        

In [45]:
from pyspark.sql.functions import length
data_df = df.withColumn('length', length(df['colorway']))
data_df.show()

+------+---------+--------------------+----------+------+
| brand|   gender|            colorway|priceGroup|length|
+------+---------+--------------------+----------+------+
|  Nike|    child|Black University ...|      <100|    26|
|  Nike|      men|    White Game Royal|   100-150|    16|
|Jordan|    child|White Tech Grey B...|   150-200|    30|
|Jordan|preschool|White Tech Grey B...|   150-200|    30|
|Jordan|      men|White Tech Grey B...|   150-200|    30|
|Jordan|  toddler|White Tech Grey B...|   150-200|    30|
|Jordan|      men|Black White-Light...|   150-200|    28|
|Jordan|    women| White Bright Citrus|   150-200|    19|
|Jordan|      men|White White-Black...|   150-200|    29|
|Jordan|      men|Hyper Royal Light...|   150-200|    34|
|Jordan|    women|Sail Rust Pink-Wh...|   150-200|    28|
|Jordan|preschool|Varsity Red Black...|      <100|    23|
|Jordan|  toddler|Varsity Red Black...|      <100|    23|
|Jordan|      men|Varsity Red Black...|   150-200|    23|
|Jordan|    ch

In [46]:
priceGroup_to_num = StringIndexer(inputCol='priceGroup',outputCol='label')
tokenizer = Tokenizer(inputCol="colorway", outputCol="token_text")
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [47]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [48]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[priceGroup_to_num, tokenizer, hashingTF, idf, clean_up])

In [49]:
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [51]:
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[10564,61...|
|  0.0|(262145,[17821,75...|
|  2.0|(262145,[10564,75...|
|  2.0|(262145,[10564,75...|
|  2.0|(262145,[10564,75...|
|  2.0|(262145,[10564,75...|
|  2.0|(262145,[78526,78...|
|  2.0|(262145,[75571,16...|
|  2.0|(262145,[75571,10...|
|  2.0|(262145,[17821,22...|
|  2.0|(262145,[23424,58...|
|  1.0|(262145,[10564,78...|
|  1.0|(262145,[10564,78...|
|  2.0|(262145,[10564,78...|
|  0.0|(262145,[110937,1...|
|  0.0|(262145,[10564,78...|
|  2.0|(262145,[75571,84...|
|  0.0|(262145,[47758,75...|
|  2.0|(262145,[8336,154...|
|  1.0|(262145,[78955,26...|
+-----+--------------------+
only showing top 20 rows



In [52]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [53]:
test_results = predictor.transform(testing)
test_results.show(5)

+-----+------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|brand|gender|            colorway|priceGroup|length|label|          token_text|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+-----+------+--------------------+----------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|ASICS|   men|         Birch Birch|   150-200|    11|  2.0|      [birch, birch]|(262144,[66585],[...|(262144,[66585],[...|(262145,[66585,26...|[-174.45526778929...|[0.99998480721538...|       0.0|
|ASICS|   men|         Black Black|   200-250|    11|  3.0|      [black, black]|(262144,[154828],...|(262144,[154828],...|(262145,[154828,2...|[-33.032389382152...|[0.94433452448477...|       0.0|
|ASICS|   men| 

In [54]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.344662
